## 3. Collaborative Filtering(협업 필터링: 사용자 리뷰 기반)

In [1]:
!pip install scikit-surprise

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-win_amd64.whl size=1122518 sha256=6ac65777ebd0403666be3773b3f0f4c7f679f082d98e50df5b094280ef84728d
  Stored in directory: c:\users\hyj\appdata\local\pip\cache\wheels\e0\44\15\6d6010d88d0e8e3694643a009f445df00a74c79c938e2c0dd4
Successfully built scikit-surprise


In [2]:
import surprise
surprise.__version__

'1.1.3'

In [4]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [5]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()
# 참고 사이트 : https://surprise.readthedocs.io/en/stable/reader.html

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings['rating'].min()

0.5

In [7]:
ratings['rating'].max()

5.0

In [8]:
reader = Reader(rating_scale=(0.5, 5))

In [10]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader=reader)
data

In [11]:
svd = SVD(random_state=0)

In [12]:
cross_validate(svd, data, measures=['RMSE','MAE'], cv=5, verbose=True) # 교차검증(딥러닝)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9022  0.8903  0.8973  0.9023  0.8861  0.8956  0.0064  
MAE (testset)     0.6946  0.6834  0.6907  0.6950  0.6805  0.6888  0.0059  
Fit time          0.96    0.96    0.99    0.99    0.99    0.98    0.01    
Test time         0.18    0.15    0.15    0.11    0.11    0.14    0.03    


{'test_rmse': array([0.90217978, 0.89028916, 0.8972504 , 0.90226403, 0.88614648]),
 'test_mae': array([0.69463754, 0.68337765, 0.6907449 , 0.69499138, 0.6804939 ]),
 'fit_time': (0.9637894630432129,
  0.9605462551116943,
  0.9937593936920166,
  0.989365816116333,
  0.9934344291687012),
 'test_time': (0.17649459838867188,
  0.14960503578186035,
  0.15455365180969238,
  0.11368393898010254,
  0.10970807075500488)}

교차 검증(k-fold 교차검증)

100개 데이터
A:1~20
B:21~40
C:41~60
D:61~80
E:81~100

ABCD(train set) E (test set)
ABCE(train set) D (test set)
ABDE(train set) C (test set)
ACDE(train set) B (test set)
BCDE(train set) A (test set)

In [13]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [14]:
ratings[ratings['userId']==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [18]:
svd.predict(1, 1029, 3) # userid =1 번인 사람이 movieid=1029인 영화에 대해서 실제 평가 3점일 때 예측 평가점수?

Prediction(uid=1, iid=1029, r_ui=3, est=2.8814455446761933, details={'was_impossible': False})

In [19]:
ratings[ratings['userId']==100]

,userId,movieId,rating,timestamp
15273,100,1,4.0,854193977
15274,100,3,4.0,854194024
15275,100,6,3.0,854194023
15276,100,7,3.0,854194024
15277,100,25,4.0,854193977
15278,100,32,5.0,854193977
15279,100,52,3.0,854194056
15280,100,62,3.0,854193977
15281,100,86,3.0,854194208
15282,100,88,2.0,854194208


In [21]:
svd.predict(100, 1073, 5) # userid =100 번인 사람이 movieid=1073인 영화에 대해서 실제 평가 5점일 때 예측 평가점수?

Prediction(uid=100, iid=1073, r_ui=5, est=4.025487270166282, details={'was_impossible': False})